In [ ]:
목표5 : batch 처리
    
목표4 : 파일/라벨의 수를 실제처럼 확대

목표3 : pooling, dropout을 넣어보자

목표2 : crossentroy를 해서 loss를 줄이는 처리

목표1 : 이미지를 하나 읽어 들이고 conv layer 2번 + fc layer 1번 처리

In [1]:
import tensorflow as tf
import os
from PIL import Image

In [2]:
image_dir = 'c:/tmp/Temp_data_Set/Test_Dataset_png/'
image_list = os.listdir(image_dir)
image_list.sort()    # sort()는 return이 없음

for i in range(len(image_list)):
    image_list[i] = image_dir + image_list[i]
    if i%1000 == 0:
        print(image_list[i])

label_dir = 'c:/tmp/Temp_data_Set/Test_Dataset_csv/label.csv'
label_list = [label_dir]
print(label_list)

imagename_queue = tf.train.string_input_producer(image_list)
labelname_queue = tf.train.string_input_producer(label_list)

c:/tmp/Temp_data_Set/Test_Dataset_png/Face00001.png
c:/tmp/Temp_data_Set/Test_Dataset_png/Face01001.png
c:/tmp/Temp_data_Set/Test_Dataset_png/Face02001.png
c:/tmp/Temp_data_Set/Test_Dataset_png/Face03001.png
c:/tmp/Temp_data_Set/Test_Dataset_png/Face04001.png
c:/tmp/Temp_data_Set/Test_Dataset_png/Face05001.png
c:/tmp/Temp_data_Set/Test_Dataset_png/Face06001.png
c:/tmp/Temp_data_Set/Test_Dataset_png/Face07001.png
['c:/tmp/Temp_data_Set/Test_Dataset_csv/label.csv']


In [3]:
image_width = 49
image_height = 61

In [4]:
image_reader = tf.WholeFileReader()
label_reader = tf.TextLineReader()
image_key, image_value = image_reader.read(imagename_queue)
label_key, label_value = label_reader.read(labelname_queue)

In [5]:
image_decoded = tf.cast(tf.image.decode_png(image_value), tf.float32)
label_decoded = tf.cast(tf.decode_csv(label_value, record_defaults=[[0]]), tf.float32)
#print(image)

label = tf.reshape(label_decoded,[1])
image = tf.reshape(image_decoded,[image_height, image_width, 1])

x, y_ = tf.train.shuffle_batch(tensors=[image, label], batch_size=32, num_threads=4, capacity=5000, min_after_dequeue=100)
    

# image를 decode하면 tensor로 저장되어 feed없이 사용하기 위해 그대로 type만 변환
#x = tf.cast(batch_image, tf.float32)
#y_ = tf.cast(batch_label, tf.float32)
#y_ = tf.reshape(y_, [-1, 1]) # batch에 맞춰서 x 하고 shape를 맞춰준다. 이거를 안하면 run에서 오류

In [6]:
# 데이터가 1개이므로 feed를 생략 x = tf.placeholder(tf.float32, shape=[None, image_width, image_height])
#y_ = tf.placeholder(tf.float32, shape=[None, 1])

In [7]:
# hidden1 layer convolution
w_hidden1 = tf.Variable(tf.truncated_normal([5,5,1,32]))
b_hidden1 = tf.Variable(tf.zeros([32]))

x_image = tf.reshape(x, [-1, image_width, image_height, 1])

h_conv1 = tf.nn.relu(tf.nn.conv2d(x_image, w_hidden1, strides=[1, 1, 1, 1], padding="SAME") + b_hidden1)
#h_pool1 = tf.nn.max_pool(h_conv1, ksize=[1, 2, 2, 1], strides=[1, 2, 2, 1], padding="SAME")
h_pool1 = tf.nn.max_pool(h_conv1, ksize=[1, 2, 2, 1], strides=[1, 1, 1, 1], padding="SAME")

In [8]:
# hidden2 layer convolution
w_hidden2 = tf.Variable(tf.truncated_normal([5,5,32,64]))
b_hidden2 = tf.Variable(tf.zeros([64]))

h_conv2 = tf.nn.relu(tf.nn.conv2d(h_pool1, w_hidden2, strides=[1, 1, 1, 1], padding="SAME") + b_hidden2)
h_pool2 = tf.nn.max_pool(h_conv2, ksize=[1, 2, 2, 1], strides=[1, 1, 1, 1], padding="SAME")

In [9]:
# fc1 layer fully-conncected
fc_w = tf.Variable(tf.truncated_normal([image_width*image_height*64, 10]))
fc_b = tf.Variable(tf.zeros([10]))

h_flat = tf.reshape(h_pool2, [-1, image_width*image_height*64])

h_fc1 = tf.nn.relu(tf.matmul(h_flat, fc_w) + fc_b)

keep_prob = tf.placeholder(tf.float32)
drop_fc = tf.nn.dropout(h_fc1, keep_prob)
#drop_fc = tf.nn.dropout(h_fc1, 0.7)

In [10]:
w_out = tf.Variable(tf.truncated_normal([10, 1]))
b_out = tf.Variable(tf.zeros([1]))

pred = tf.matmul(drop_fc, w_out) + b_out

In [11]:
print(pred)
print(y_)

Tensor("add_3:0", shape=(32, 1), dtype=float32)
Tensor("shuffle_batch:1", shape=(32, 1), dtype=float32)


In [12]:
loss = tf.reduce_min(tf.nn.sigmoid_cross_entropy_with_logits(labels=y_, logits=pred))
train = tf.train.AdamOptimizer(1e-4).minimize(loss)

In [13]:
# prediction한 값과 label값을 비교 : True/False 리스트
correct_prediction = tf.equal(tf.argmax(pred,1), tf.argmax(y_,1))

# True/False 리스트를 실수형으로 casting한다.
accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))

In [14]:
with tf.Session() as sess:
    print("started")
    coord = tf.train.Coordinator()
    thread = tf.train.start_queue_runners(sess=sess, coord=coord)
    
    sess.run(tf.global_variables_initializer())

    for i in range(100):
#        print(sess.run(image_key))
#        print(sess.run(image_key))
        sess.run(train, {keep_prob: 0.7})
#        sess.run(train)
        
        #if train==True:
        _looss = sess.run(loss, {keep_prob: 0.7})
        _accuracy = sess.run(accuracy, {keep_prob: 0.7})
        #if release_mode==Train:
        #result = sess.run(pred, {keep_prob: 1.0})
        
        print("-----------------")
        print("loss: ", _looss)
        print("accuracy: ", _accuracy)
        
#    print(image)

#    Image.fromarray(image).show()

    print("training done")
    
    coord.request_stop()
    coord.join(thread)

started
-----------------
loss:  -9.29883e+07
accuracy:  1.0
-----------------
loss:  -6.03188e+07
accuracy:  1.0
-----------------
loss:  -1.73295e+08
accuracy:  1.0
-----------------
loss:  -1.22344e+08
accuracy:  1.0
-----------------
loss:  -1.72406e+07
accuracy:  1.0
-----------------
loss:  -8.70677e+07
accuracy:  1.0
-----------------
loss:  -1.12025e+08
accuracy:  1.0
-----------------
loss:  -7.08827e+07
accuracy:  1.0
-----------------
loss:  -4.16517e+08
accuracy:  1.0
-----------------
loss:  -2.34204e+08
accuracy:  1.0
-----------------
loss:  -1.21609e+08
accuracy:  1.0
-----------------
loss:  -1.85732e+08
accuracy:  1.0
-----------------
loss:  -2.95414e+08
accuracy:  1.0
-----------------
loss:  -2.25075e+08
accuracy:  1.0
-----------------
loss:  -1.66845e+08
accuracy:  1.0
-----------------
loss:  -3.39159e+08
accuracy:  1.0
-----------------
loss:  -1.71385e+08
accuracy:  1.0
-----------------
loss:  -4.77425e+08
accuracy:  1.0
-----------------
loss:  -1.91557e+08
